<a href="https://colab.research.google.com/github/YugantGotmare/NLP/blob/master/Spam_or_Ham_classifier_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
df = pd.read_csv('/content/SMSSpamCollection', sep='\t',
                           names=["label", "message"])

In [22]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5572 non-null   object
 1   message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [24]:
df.isnull().sum()

label      0
message    0
dtype: int64

In [25]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [26]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [27]:
cv = CountVectorizer(max_features=2500)
x = cv.fit_transform(corpus).toarray()

y=pd.get_dummies(df['label'])
y=y.iloc[:,1].values

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)

In [29]:
spam_detect_model = MultinomialNB()

In [30]:
spam_detect_model.fit(x_train, y_train)

MultinomialNB()

In [31]:
y_pred=spam_detect_model.predict(x_test)

In [34]:
from sklearn.metrics import confusion_matrix, accuracy_score

result = confusion_matrix(y_test,y_pred)
result

array([[946,   9],
       [  7, 153]])

In [36]:
final_accuracy = accuracy_score(y_test,y_pred)
final_accuracy

0.9856502242152466

In [44]:
new_email = "Please reply to get this offer"
new_email_transformed = cv.transform([new_email])

new_email_pred = spam_detect_model.predict(new_email_transformed)
print("Class label prediction:", new_email_pred[0])

if new_email_pred == 1:
  print("Ham")
else:
  print("spam")

Class label prediction: 0
spam
